In [173]:
import torch
import torch.nn as nn

In [174]:
#input = torch.randn(1, 512, 10, 10)
#print(input.size())
#input = input.view(input.size()[0], -1)
#print(input.size())

In [175]:
#num_class = 1000
#fc = nn.Linear(512, num_class)
#fc = nn.Linear(512, num_class)

In [176]:
#output = fc(input)
#print(output.size())

In [177]:
from resnet_org import *

In [178]:
resnet18_8s = resnet18(pretrained=True, fully_conv=True, output_stride=8, remove_avg_pool_layer=True)

# model に渡す入力を作成

In [179]:
import numpy as np
h, w = 256, 256
#img = np.random.uniform(-1, 1, [1, 3, h, w]).astype(np.float32)
img = np.random.uniform(-1, 1, [h, w, 3]).astype(np.float32)

In [180]:
#from matplotlib import pyplot as plt
#%matplotlib inline

#plt.imshow(img)
#plt.show()

In [181]:
# model に渡す配列に変換

#print("before resize: {}".format(img))
img_rs = np.reshape(img, (1, 3, h, w))
#print(img_rs)
img_tensor = torch.tensor(img_rs)

In [182]:
#out = resnet18_8s(img_tensor)
#print(out)

## FCN 作成用に ResNet の出力層と GAP 層 を取り除く

In [183]:
# ロードした ResNet の最終2層
#print(*list(resnet18_8s.children())[-2:])

# ロードした ResNetの GAP 層と FC層を取り除く
# 注！ロードしたResNetを別変数に代入した場合、x2sなど、途中から出力を取り出すことができなくなる。
#resnet = nn.Sequential(*list(resnet18_8s.children())[:-2])

#print(resnet)

## アップサンプリング部分の設計

In [184]:
fcdim = 256
s16dim = 256
s8dim = 128
s4dim = 64
s2dim = 32
raw_dim = 32
ver_dim = 2
seg_dim = 2

In [185]:
# print(resnet18_8s.inplanes) -> 512
# ResNetの FC層を置き換えるための層
fc = nn.Sequential(
    nn.Conv2d(
        resnet18_8s.inplanes,
        fcdim,
        kernel_size = 3,
        stride = 1,
        padding = 1,
        bias=False),
    nn.BatchNorm2d(fcdim),
    nn.ReLU(True),
 )

# x16s -> 256
conv16s = nn.Sequential(
    nn.Conv2d(256+ fcdim, s16dim, 3, 1, 1, bias=False),
    nn.BatchNorm2d(s16dim),
    nn.LeakyReLU(0.1, True),
)
up32sTo16s = nn.UpsamplingBilinear2d(scale_factor=2)

# x8s -> 128
conv8s = nn.Sequential(
    nn.Conv2d(128 + fcdim, s8dim, 3, 1, 1, bias=False),
    nn.BatchNorm2d(s8dim),
    nn.LeakyReLU(0.1, True),
)
up16sTo8s = nn.UpsamplingBilinear2d(scale_factor=2)

# x4s -> 64
conv4s = nn.Sequential(
    nn.Conv2d(64 + s8dim, s4dim, 3, 1, 1, bias=False),
    nn.BatchNorm2d(s4dim),
    nn.LeakyReLU(0.1, True),
)
up8sTo4s = nn.UpsamplingBilinear2d(scale_factor=2)

# x2s -> 64
conv2s = nn.Sequential(
    nn.Conv2d(64 + s4dim, s2dim, 3, 1, 1, bias=False),
    nn.BatchNorm2d(s2dim),
    nn.LeakyReLU(0.1, True),
)
up4sTo2s = nn.UpsamplingBilinear2d(scale_factor=2)

convraw = nn.Sequential(
    nn.Conv2d(3 + s2dim, raw_dim, 3, 1, 1, bias=False),
    nn.BatchNorm2d(raw_dim),
    nn.LeakyReLU(0.1, True),
    nn.Conv2d(raw_dim, seg_dim + ver_dim, 1, 1),
)
up2sToraw = nn.UpsamplingBilinear2d(scale_factor=2)

## ResNet の FC 層を置き換える

In [186]:
resnet18_8s.fc = fc

In [187]:
x = img_tensor
#x2s, x4s, x8s, x16s, x32s = resnet(x)
x2s, x4s, x8s, x16s, x32s, xfc = resnet18_8s(x)

print("x2s output = {}".format(x2s.size()))
print("x4s output = {}".format(x4s.size()))
print("x8s output = {}".format(x8s.size()))
print("x16s output = {}".format(x16s.size()))
print("x32s output = {}".format(x32s.size()))
print("xfc output = {}".format(xfc.size()))

x2s output = torch.Size([1, 64, 128, 128])
x4s output = torch.Size([1, 64, 64, 64])
x8s output = torch.Size([1, 128, 32, 32])
x16s output = torch.Size([1, 256, 16, 16])
x32s output = torch.Size([1, 512, 8, 8])
xfc output = torch.Size([1, 256, 8, 8])


In [188]:
#up16sTo8s  = nn.UpsamplingBilinear2d(scale_factor=2)

#fm = up16sTo8s(xfc)
#torch.cat([fm, x16s], 1)
#print(fm.size())

In [189]:
print("xfc output = {}".format(xfc.size()))
fm = up32sTo16s(xfc)
fm = conv16s(torch.cat([fm, x16s], 1))

print("32s -> 16s output = {}".format(fm.size()))
fm = up16sTo8s(fm)
fm = conv8s(torch.cat([fm, x8s], 1))

print("16s -> 8s output = {}".format(fm.size()))
fm = up8sTo4s(fm)
fm = conv4s(torch.cat([fm, x4s], 1))

fm = up4sTo2s(fm)
fm = conv2s(torch.cat([fm, x2s], 1))

fm = up2sToraw(fm)
x = convraw(torch.cat([fm, x], 1))
seg_pred = x[:, : seg_dim, :, :]
ver_pred = x[:, seg_dim :, :, :]

xfc output = torch.Size([1, 256, 8, 8])
32s -> 16s output = torch.Size([1, 256, 16, 16])
16s -> 8s output = torch.Size([1, 128, 32, 32])
